In [1]:
import sqlite3
import yaml

# Wrappers

In [2]:
def build_create_query(table_name, table_scheme_dict):
    if len(table_scheme_dict["primary_keys"]) > 0:
        a = ", ".join([f"{k} {v}" for k,v in table_scheme_dict["columns"].items()])
        b = ", ".join([f"{v}" for v in table_scheme_dict["primary_keys"]])

        res = f"create table if not exists {table_name} ({a}, PRIMARY KEY ({b}))"
    else:
        a = ", ".join([f"{k} {v}" for k,v in table_scheme_dict["columns"].items()])
        
        res = f"create table if not exists {table_name} ({a})"
    return res

def build_insert_query(table_name, db_schema_dict):
    query_str = f"INSERT OR IGNORE INTO {table_name}" 
    query_str += "(" + ", ".join([k for k in db_schema_dict[table_name]["columns"]]) + ")"
    query_str += " values "
    query_str += "(" + ", ".join(["?" for i in db_schema_dict[table_name]["columns"]]) + ")"
    return query_str

In [3]:
with open("example_db.yml", "r") as fp:
    new_db_schema = yaml.safe_load(fp)

In [4]:
new_db_schema

{'UsersTable': {'columns': {'ID': 'UInt64',
   'UserGroup': 'UInt2',
   'Name': 'String'},
  'primary_keys': ['UserGroup']},
 'MessagesTable': {'columns': {'ID': 'UInt64',
   'UserID': 'UInt64',
   'Caption': 'String',
   'CreatedAt': 'DateTime',
   'UpdatedAt': 'DateTime'},
  'primary_keys': []}}

# Usage

In [5]:
conn = sqlite3.connect("demo.db")
cur = conn.cursor()

## create db from empty state

In [6]:
for sch in new_db_schema:
    query = build_create_query(sch, new_db_schema[sch])
    cur.execute(query)

## fill db with stuff

In [11]:
temp_data = list()
temp_data.append([0, 1, "John Doe"])

In [12]:
cur.executemany(build_insert_query("UsersTable", new_db_schema), temp_data)
conn.commit()

In [13]:
cur.execute("select * from UsersTable").fetchall()

[(0, 1, 'John Doe')]